In [370]:
import pandas as pd

# Load your dataset into a pandas DataFrame
df1 = pd.read_csv('~/temp/sets_dist.csv', sep=";")
df1 = df1.sample(frac=1, random_state=42)

df1



,id,person,class,number
9,S2_file_58_5_1036_1065.npz,S2,5,29
164,S1_file_28_9_1750_1837.npz,S1,9,87
139,S3_file_87_10_533_599.npz,S3,10,66
46,S2_file_20_8_946_1320.npz,S2,8,374
94,S3_file_184_1_323_346.npz,S3,1,23
...,...,...,...,...
188,S1_file_21_8_1845_2262.npz,S1,8,417
71,S1_file_36_5_1081_1126.npz,S1,5,45
106,S2_file_20_3_463_492.npz,S2,3,29
270,S2_file_27_10_205_324.npz,S2,10,119


In [371]:
# Group the DataFrame by 'person' and 'class', then sum the 'number' column within each group
df2 = df1.groupby(['person', 'class'])['number'].sum().reset_index()

# Rename the 'number' column to 'frames_per_person_class'
df2.rename(columns={'number': 'frames_per_person_class'}, inplace=True)

# Display the resulting DataFrame
print(df2)

   person  class  frames_per_person_class
0      S1      1                     1997
1      S1      2                     1063
2      S1      3                      620
3      S1      4                     5530
4      S1      5                      422
5      S1      6                     1637
6      S1      7                      488
7      S1      8                     2025
8      S1      9                      739
9      S1     10                     1433
10     S1     11                      138
11     S2      1                     1073
12     S2      2                      355
13     S2      3                      374
14     S2      4                     5698
15     S2      5                      202
16     S2      6                      506
17     S2      7                      882
18     S2      8                     4633
19     S2      9                      853
20     S2     10                     6025
21     S2     11                      582
22     S3      1                  

In [372]:
# Calculate the new columns
df2['train'] = df2['frames_per_person_class'] * 0.9
df2['val'] = df2['frames_per_person_class'] * 0.1
df2['test'] = df2['frames_per_person_class'] * 0.1

# Print the updated DataFrame
print(df2)

   person  class  frames_per_person_class   train    val   test
0      S1      1                     1997  1797.3  199.7  199.7
1      S1      2                     1063   956.7  106.3  106.3
2      S1      3                      620   558.0   62.0   62.0
3      S1      4                     5530  4977.0  553.0  553.0
4      S1      5                      422   379.8   42.2   42.2
5      S1      6                     1637  1473.3  163.7  163.7
6      S1      7                      488   439.2   48.8   48.8
7      S1      8                     2025  1822.5  202.5  202.5
8      S1      9                      739   665.1   73.9   73.9
9      S1     10                     1433  1289.7  143.3  143.3
10     S1     11                      138   124.2   13.8   13.8
11     S2      1                     1073   965.7  107.3  107.3
12     S2      2                      355   319.5   35.5   35.5
13     S2      3                      374   336.6   37.4   37.4
14     S2      4                     569

In [373]:


# Function to update df2 based on df1
def update_df2(row):
    person = row['person']
    class_val = row['class']
    frames = row['number']
    
    matching_rows = df2[(df2['person'] == person) & (df2['class'] == class_val)]
    if not matching_rows.empty:
        idx = matching_rows.index[0]
    if (df2.at[idx, 'train']-frames) >=  -900:
        df2.at[idx, 'train'] -= frames
        row['set'] = 'train'
    elif (df2.at[idx, 'val']-frames) >= -200:
        df2.at[idx, 'val'] -= frames
        row['set'] = 'val'
    elif (df2.at[idx, 'test']-frames) >= -1000:
        df2.at[idx, 'test'] -= frames
        row['set'] = 'test'    
    
    return row  # Return the updated row from df2

# Apply the function to each row in df1
df1 = df1.apply(update_df2, axis=1)

print(df2)



   person  class  frames_per_person_class  train    val   test
0      S1      1                     1997 -199.7  199.7  199.7
1      S1      2                     1063 -106.3  106.3  106.3
2      S1      3                      620  -62.0   62.0   62.0
3      S1      4                     5530 -553.0  553.0  553.0
4      S1      5                      422  -42.2   42.2   42.2
5      S1      6                     1637 -163.7  163.7  163.7
6      S1      7                      488  -48.8   48.8   48.8
7      S1      8                     2025 -202.5  202.5  202.5
8      S1      9                      739  -73.9   73.9   73.9
9      S1     10                     1433 -143.3  143.3  143.3
10     S1     11                      138  -13.8   13.8   13.8
11     S2      1                     1073 -107.3  107.3  107.3
12     S2      2                      355  -35.5   35.5   35.5
13     S2      3                      374  -37.4   37.4   37.4
14     S2      4                     5698 -569.8  569.8

In [374]:
print(df1)

                             id person  class  number    set
9    S2_file_58_5_1036_1065.npz     S2      5      29  train
164  S1_file_28_9_1750_1837.npz     S1      9      87  train
139   S3_file_87_10_533_599.npz     S3     10      66  train
46    S2_file_20_8_946_1320.npz     S2      8     374  train
94    S3_file_184_1_323_346.npz     S3      1      23  train
..                          ...    ...    ...     ...    ...
188  S1_file_21_8_1845_2262.npz     S1      8     417  train
71   S1_file_36_5_1081_1126.npz     S1      5      45  train
106    S2_file_20_3_463_492.npz     S2      3      29  train
270   S2_file_27_10_205_324.npz     S2     10     119  train
102    S1_file_10_1_322_499.npz     S1      1     177  train

[329 rows x 5 columns]


In [375]:
df1 = df1.apply(update_df2, axis=1)

In [376]:
print(df2)

   person  class  frames_per_person_class  train    val   test
0      S1      1                     1997 -858.7 -149.3 -789.3
1      S1      2                     1063 -879.3 -183.7  106.3
2      S1      3                      620 -682.0   62.0   62.0
3      S1      4                     5530 -802.0   74.0 -977.0
4      S1      5                      422 -464.2   42.2   42.2
5      S1      6                     1637 -837.7 -181.3 -454.3
6      S1      7                      488 -536.8   48.8   48.8
7      S1      8                     2025 -893.5 -198.5 -730.5
8      S1      9                      739 -812.9   73.9   73.9
9      S1     10                     1433 -894.3 -162.7 -232.7
10     S1     11                      138 -151.8   13.8   13.8
11     S2      1                     1073 -885.3 -187.7  107.3
12     S2      2                      355 -390.5   35.5   35.5
13     S2      3                      374 -411.4   37.4   37.4
14     S2      4                     5698 -809.8 -182.2

In [377]:
print(df1)

                             id person  class  number    set
9    S2_file_58_5_1036_1065.npz     S2      5      29  train
164  S1_file_28_9_1750_1837.npz     S1      9      87  train
139   S3_file_87_10_533_599.npz     S3     10      66  train
46    S2_file_20_8_946_1320.npz     S2      8     374  train
94    S3_file_184_1_323_346.npz     S3      1      23  train
..                          ...    ...    ...     ...    ...
188  S1_file_21_8_1845_2262.npz     S1      8     417   test
71   S1_file_36_5_1081_1126.npz     S1      5      45  train
106    S2_file_20_3_463_492.npz     S2      3      29  train
270   S2_file_27_10_205_324.npz     S2     10     119  train
102    S1_file_10_1_322_499.npz     S1      1     177   test

[329 rows x 5 columns]


In [378]:
nan_rows_df1 = df1[df1['set'].isna()]
print(nan_rows_df1)


Empty DataFrame
Columns: [id, person, class, number, set]
Index: []


In [379]:
# Calculate the percentages of rows with sets "a", "b", and "c"
set_counts = df1['set'].value_counts()
total_rows = len(df1)
percentages = (set_counts / total_rows) * 100

print(percentages)

set
train    78.723404
test     11.854103
val       9.422492
Name: count, dtype: float64


In [380]:
df1

,id,person,class,number,set
9,S2_file_58_5_1036_1065.npz,S2,5,29,train
164,S1_file_28_9_1750_1837.npz,S1,9,87,train
139,S3_file_87_10_533_599.npz,S3,10,66,train
46,S2_file_20_8_946_1320.npz,S2,8,374,train
94,S3_file_184_1_323_346.npz,S3,1,23,train
...,...,...,...,...,...
188,S1_file_21_8_1845_2262.npz,S1,8,417,test
71,S1_file_36_5_1081_1126.npz,S1,5,45,train
106,S2_file_20_3_463_492.npz,S2,3,29,train
270,S2_file_27_10_205_324.npz,S2,10,119,train


In [385]:
# Sort df1 by index
df1 = df1.sort_index()

# Specify the file path for the CSV file
csv_file_path = "~/temp/sets.txt"

# Select and save only the "ID" and "set" columns to a text file (TSV format)
df1[['id', 'set']].to_csv(csv_file_path, sep='\t', index=False)

# Print a message to confirm the file was saved
print(f"DataFrame df1 has been reordered and saved to {csv_file_path}.")

DataFrame df1 has been reordered and saved to ~/temp/sets.txt.


In [386]:
def read_csv(file_path):
    split_info = []
    with open(file_path, 'r') as file:
        for line in file:
            print(line)
            video_name, split = line.strip().split('\t')
            split_info.append({'video_name': video_name, 'split': split})
    return split_info

read_csv('/data/iballester/datasets/BAD/sets.txt')

ValueError: too many values to unpack (expected 2)